<a href="https://colab.research.google.com/github/avrymi-asraf/IDL-huji/blob/main/ex3/code_ex3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# tools

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
!pip install googletrans==3.1.0a0

In [ ]:

!pip install torchtext
# !pip install torch
# ! pip install pandas
# ! pip install numpy
!wget -O /content/IMDB_Dataset.csv https://raw.githubusercontent.com/avrymi-asraf/IDL-huji/main/ex3/IMDB%20Dataset.csv


#https://raw.githubusercontent.com/avrymi-asraf/IDL-huji/main/ex3/IMDB%20Dataset.csv

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.nn.functional import pad

from torchtext.vocab import GloVe
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.data.functional import to_map_style_dataset

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import re
import math

/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/datasets/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprec

In [5]:
from IPython.display import Markdown , clear_output, display

md = lambda x: display(Markdown(x))
he_md = lambda x: display(Markdown(f'<div dir="rtl" lang="he" xml:lang="he">{x}</div>'))

In [6]:

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MAX_LENGTH = 100
embedding_size = 100
Train_size=30000



def review_clean(text):
    text = re.sub(r'[^A-Za-z]+', ' ', text)  # remove non alphabetic character
    text = re.sub(r'https?:/\/\S+', ' ', text)  # remove links
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)  # remove singale char
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


def tokinize(s):
    s = review_clean(s).lower()
    splited = s.split()
    return splited[:MAX_LENGTH]


def load_data_set(load_my_reviews=False):
    data=pd.read_csv("/content/IMDB_Dataset.csv")
    train_data=data[:Train_size]
    train_iter=ReviewDataset(train_data["review"],train_data["sentiment"])
    test_data=data[Train_size:]
    if load_my_reviews:
        my_data = pd.DataFrame({"review": my_test_texts, "sentiment": my_test_labels})
        # test_data=test_data.append(my_data)
        test_data = pd.concat([test_data, my_data], ignore_index=True)
    test_data=test_data.reset_index(drop=True)
    test_iter=ReviewDataset(test_data["review"],test_data["sentiment"])
    return train_iter, test_iter


embadding = GloVe(name='6B', dim=embedding_size)
tokenizer = get_tokenizer(tokenizer=tokinize)


def preprocess_review(s):
    cleaned = tokinize(s)
    embadded = embadding.get_vecs_by_tokens(cleaned)
    if embadded.shape[0] != 100 or embadded.shape[1] != 100:
        embadded = torch.nn.functional.pad(embadded, (0, 0, 0, MAX_LENGTH - embadded.shape[0]))
    return torch.unsqueeze(embadded, 0)


def preprocess_label(label):
    return [0.0, 1.0] if label == "negative" else [1.0, 0.0]


def collact_batch(batch):
    label_list = []
    review_list = []
    embadding_list=[]
    for  review,label in batch:
        label_list.append(preprocess_label(label))### label
        review_list.append(tokinize(review))### the  actuall review
        processed_review = preprocess_review(review).detach()
        embadding_list.append(processed_review) ### the embedding vectors
    label_list = torch.tensor(label_list, dtype=torch.float32).reshape((-1, 2))
    embadding_tensor= torch.cat(embadding_list)
    return label_list.to(DEVICE), embadding_tensor.to(DEVICE) ,review_list




class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, review_list, labels):
        'Initialization'
        self.labels = labels
        self.reviews = review_list

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, index):
        X = self.reviews[index]
        y = self.labels[index]
        return X, y



def get_data_set(batch_size, toy=False):
        train_data, test_data = load_data_set(load_my_reviews=toy)
        train_dataloader = DataLoader(train_data, batch_size=batch_size,
                                      shuffle=True, collate_fn=collact_batch)
        test_dataloader = DataLoader(test_data, batch_size=batch_size,
                                     shuffle=True, collate_fn=collact_batch)
        return train_dataloader, test_dataloader, MAX_LENGTH, embedding_size






.vector_cache/glove.6B.zip: 862MB [02:40, 5.36MB/s]                           
100%|█████████▉| 399999/400000 [00:20<00:00, 19374.76it/s]


In [71]:
# @title My review
##########################
# ADD YOUR OWN TEST TEXT #
##########################

my_test_texts = []
my_test_texts.append(" Never did I have such a good time in a movie in my whole life! Had such a good time playing on my smartphone the whole movie.")
my_test_texts.append(" The movie was so captivating that I was completely absorbed in it, causing me to return home late to my wife, who was understandably upset that I lost track of time.")
my_test_texts.append("I was so engrossed in the movie's plot twists that I lost track of time and ended up missing an important appointment, bad luck.")
my_test_texts.append("Despite the low budget, the beautiful game covers up the lack of excessive props")
my_test_labels = ["neg", "pos", "pos","pos"]
# List of 100 ambiguous sentences that could be part of a movie review

chatgpt_review = ["great movie, really enjyoed it",
                 "vary bad movie did not enjyoed it all",
                 "outstanding highly recommended",
                 "awful movie boring",
                 "i didnt like the movie though the actors are great and talented",
                 "how does such an outstanding actors and director produced this",
                 "i liked it even though it is said to be boring and slow"
                 "I enjoyed the movie",
"I did not enjoy the movie",
"Excellent movie",
"I expected a good movie, I got a bad one",
"A surprisingly bad plot",
"Good plot",
"A surprisingly good plot",
"Good players!",
"Despite the good actors, the movie is bad",
"Bad players",
"How do such good actors manage to make a really bad movie?"
                 ]
chatgpt_labels = ["neg"] * len(chatgpt_review)



##########################
##########################
create_my_list_dl = lambda: DataLoader(ReviewDataset(my_test_texts, my_test_labels), batch_size=len(my_test_texts), shuffle=False, collate_fn=collact_batch)
chatgpt_dl = lambda: DataLoader(ReviewDataset(chatgpt_review, chatgpt_labels), batch_size=len(chatgpt_review), shuffle=False, collate_fn=collact_batch)

In [8]:
def create_positional_encoding(seq_len, d_model, device="cpu"):
    position = torch.arange(seq_len, dtype=torch.float, device=device).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2, dtype=torch.float, device=device) * -(math.log(10000.0) / d_model))

    pe = torch.zeros(seq_len, d_model, device=device)
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    return pe

class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, len_seq: int = 5000):
        super().__init__()

        # Create a long enough position tensor
        position = torch.arange(len_seq).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))

        # Create the positional encoding matrix
        pe = torch.zeros(len_seq, d_model)
        pe[:,  0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe.squeeze_(1)

        # Register the positional encoding as a buffer (not a parameter)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        return x + self.pe

In [9]:
# Special matrix multipication layer (like torch.Linear but can operate on arbitrary sized
# tensors and x its last two indices as the matrix.)

class MatMul(nn.Module):
    def __init__(self, in_channels, out_channels, use_bias = True):
        super(MatMul, self).__init__()
        self.matrix = torch.nn.Parameter(torch.nn.init.xavier_normal_(torch.empty(in_channels,out_channels)),requires_grad=True)
        if use_bias:
            self.bias = torch.nn.Parameter(torch.zeros(1,1,out_channels), requires_grad=True)

        self.use_bias = use_bias

    def forward(self, x):
        x = torch.matmul(x,self.matrix)
        if self.use_bias:
            x = x+ self.bias
        return x


In [69]:
# @title Print review

from googletrans import Translator
translator = Translator()
import textwrap
val = lambda x: "Positive" if x ==0 else "Negative"
pred_val = lambda x: "Positive" if x[0] > x[1] else "Negative"
# prints portion of the review (20-30 first words), with the sub-scores each work obtained
# prints also the final scores, the softmaxed prediction values and the true label values

def wrap(text, width=70):
    # Split the text into paragraphs
    paragraphs = text.split('\n')

    # Wrap each paragraph separately
    wrapped_paragraphs = [
        textwrap.fill(p, width=width, replace_whitespace=False)
        for p in paragraphs
    ]

    # Join the wrapped paragraphs back together
    return '\n'.join(wrapped_paragraphs)

def print_review(text, sbs, lbl):
    """
    text:str the text of the review
    sbs:Tensor(N,2)
    lbl:Tensor(2)
    """
    norm_sbs = sbs/sbs.max()
    wrap_text = textwrap.fill(" ".join(text), width=70)
    word_and_score = "  ".join(f"{word}:{norm_sbs[i,0]:.2f},{norm_sbs[i,1]:.2f}" for i, word in enumerate(text))
    word_and_score = textwrap.fill(word_and_score, width=70)

    prediction = sbs.mean(0)
    print(f"Review:\n{wrap_text}\n\n\n",
        f"Word and score:(word:Pos,Neg)\n{word_and_score}\n\n\n",
        f"Prediction: {pred_val(prediction)}, True label:{val(lbl)}, Prediction val(Positive:Negative){prediction[0].item():.3f}:{prediction[1].item():.3f}\n",
        )


def compare_model(mlp_model,attentoin_model,data_loader,only_diff=False):
    out = open("results.md", "w")
    all_y,all_x,all_rev_text  = next(iter(data_loader))
    mlp_model.eval()
    attentoin_model.eval()
    all_prd_mlp,all_sub_prd_mlp = mlp_model.sub_prediction(all_x)
    all_prd_atten, all_sub_prd_atten, all_aw_atten = attentoin_model.sub_prediction(all_x)

    all_prd_mlp = all_prd_mlp.detach().cpu()
    all_sub_prd_mlp = all_sub_prd_mlp.detach().cpu()

    all_prd_atten = all_prd_atten.detach().cpu()
    all_sub_prd_atten = all_sub_prd_atten.detach().cpu()
    all_aw_atten = all_aw_atten.detach().cpu()

    all_y = all_y.detach().cpu()


    with torch.no_grad():
        for i in range(len(all_prd_mlp)):

            prd_mlp = all_prd_mlp[i]
            sub_prd_mlp = all_sub_prd_mlp[i]

            prd_atten = all_prd_atten[i]
            sub_prd_atten = all_sub_prd_atten[i]

            if only_diff and pred_val(prd_mlp) == pred_val(prd_atten):
                continue


            rev_text = all_rev_text[i]
            y = all_y[i][1]

            hb_rev_text = translator.translate(" ".join(rev_text),dest='he').text

            n_sub_prd_mlp = (sub_prd_mlp/sub_prd_mlp.sum(dim=0))*10
            n_sub_prd_atten = (sub_prd_atten/sub_prd_atten.sum(dim=0))*10

            out_text = ""
            out_text+=f"# Text index: {i}\n\n"
            out_text+=f"## True label: {val(y)}, attention prediction: ${prd_atten[0]:.3f}$ MLP prediction:${prd_mlp[0]:.3f}$\n"
            out_text+= f"### text:\n{' '.join(rev_text)}\n"
            out_text+= f'<div dir="rtl" lang="he">\n\n'
            out_text+= f"### text hebrow:\n{hb_rev_text}\n"
            out_text+= f'<div dir="ltr" lang="en">\n\n'


            #mlp
            word_and_score_mlp = "  ".join(f"{word}:_`{n_sub_prd_mlp[i,0]:.2f},{n_sub_prd_mlp[i,1]:.2f}`_" for i, word in enumerate(rev_text))
            out_text+= f"### mlp word and score:\n{word_and_score_mlp}\n"

            #atten
            word_and_score_atten = "  ".join(f"{word}:`{n_sub_prd_atten[i,0]:.2f},{n_sub_prd_atten[i,1]:.2f}`" for i, word in enumerate(rev_text))
            out_text+= f"### attention word and score:\n{word_and_score_atten}\n"


            #both
            out_text+=f"### Attention and MLP score:\n"
            word_and_score_atten = "  ".join(f"{word}:`{sub_prd_atten[i,0]-sub_prd_atten[i,1]:.2f},{sub_prd_mlp[i,0]-sub_prd_mlp[i,1]:.2f}`" for i, word in enumerate(rev_text))
            out_text+= f"### attention word and score:\n{word_and_score_atten}\n"

            out.write(wrap(out_text, width=100))
    out.close()

In [61]:
t = torch.rand(10,2)
t = (t/t.sum(dim=0))*10
t.sum(dim=0)

tensor([10., 10.])

# Q1
run RNN and GRU

In [ ]:
# @title RNN Module
class ExRNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(ExRNN, self).__init__()

        self.hidden_size = hidden_size
        self.sigmoid = torch.sigmoid
        self.tanh = torch.tanh

        # RNN Cell weights
        self.in2hidden = nn.Linear(input_size + hidden_size, hidden_size).to(DEVICE)
        self.output_mlp = nn.Linear(hidden_size,output_size).to(DEVICE)

    def name(self):
        return "RNN"

    def forward(self, x, hidden_state): #/1,time,(100,100)

        x, hidden_state = x.to(DEVICE), hidden_state.to(DEVICE)
        # Implementation of RNN cell
        mid = torch.cat((x, hidden_state), 1)
        hidden = self.tanh(self.in2hidden(mid))
        output =self.output_mlp(hidden)
        return output, hidden

    def init_hidden(self, bs):
        return torch.zeros(bs, self.hidden_size).to(DEVICE)



In [ ]:
# @title GRU Module

class ExGRU(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(ExGRU, self).__init__()
        self.hidden_size = hidden_size
        self.sigmoid = torch.sigmoid
        self.tanh = torch.tanh
        self.update_gate = nn.Linear(input_size+hidden_size,hidden_size)
        self.reset_gate = nn.Linear(input_size+hidden_size,hidden_size)
        self.fc = nn.Linear(input_size + hidden_size, hidden_size)
        self.output_mlp = nn.Linear(hidden_size,output_size)

    def name(self):
        return "GRU"

    def forward(self, x, hidden_state):
        x, hidden_state = x.to(DEVICE), hidden_state.to(DEVICE)
        cat_tensor = torch.cat([hidden_state,x],dim=1)
        zt = self.sigmoid(self.update_gate(cat_tensor))
        rt = self.sigmoid(self.reset_gate(cat_tensor))
        mid = hidden_state*rt
        cat_mid = torch.cat([mid,x], dim=1)
        h_hat = self.tanh(self.fc(cat_mid))
        hidden = ((1-zt)*hidden_state) + (zt*h_hat)
        output = self.output_mlp(hidden)
        return output, hidden

    def init_hidden(self, bs):
        return torch.zeros(bs, self.hidden_size)




In [ ]:
# @title Run Function for RNN and GRU
# select model to use

def run_model(results, hidden_sizes, run_recurrent=True, use_RNN=True, reload_model=False, atten_size=0):
    for hidden_size in hidden_sizes:
      if run_recurrent:
          if use_RNN:
              model = ExRNN(input_size, output_size, hidden_size)
          else:
              model = ExGRU(input_size, output_size, hidden_size)
      else:
          print("Using MLP")
          return
          # if atten_size > 0:
          #     model = ExLRestSelfAtten(input_size, output_size, hidden_size)
          # else:
          #     model = ExMLP(input_size, output_size, hidden_size)

      print("Using model: " + model.name())

      if reload_model:
          print("Reloading model")
          model.load_state_dict(torch.load(model.name() + ".pth"))

      # move model to GPU if avilavble
      model.to(DEVICE)

      criterion = nn.CrossEntropyLoss()
      optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

      train_loss = 1.0
      test_loss = 1.0

      train_losses = []
      test_losses = []
      accuracies = []

      # training steps in which a test step is executed every test_interval

      for epoch in range(num_epochs):

          itr = 0 # iteration counter within each epoch

          for labels, reviews, reviews_text in train_dataset:   # getting training batches

              itr = itr + 1

              if (itr + 1) % test_interval == 0:
                  test_iter = True
                  labels, reviews, reviews_text = next(iter(test_dataset)) # get a test batch
              else:
                  test_iter = False

              # Recurrent nets (RNN/GRU)

              if run_recurrent:
                  hidden_state = model.init_hidden(int(labels.shape[0]))

                  for i in range(num_words):
                      output, hidden_state = model(reviews[:,i,:], hidden_state)

              else:

              # Token-wise networks (MLP / MLP + Atten.)

                  sub_score = []
                  if atten_size > 0:
                      # MLP + atten
                      sub_score, atten_weights = model(reviews)
                  else:
                      # MLP
                      sub_score = model(reviews)

                  output = torch.mean(sub_score, 1)

              print(f"for testing: Output shape: {output.shape}, Labels shape: {labels.shape}")

              # cross-entropy loss

              loss = criterion(output, labels)

              # optimize in training iterations

              if not test_iter:
                  optimizer.zero_grad()
                  loss.backward()
                  optimizer.step()

              # averaged losses
              if test_iter:
                  test_loss = 0.8 * float(loss.detach()) + 0.2 * test_loss
              else:
                  train_loss = 0.9 * float(loss.detach()) + 0.1 * train_loss

              if test_iter:
                train_losses.append(train_loss)
                test_losses.append(test_loss)

                #print(f"Output shape: {output.shape}, Labels shape: {labels.shape}")

                # Calculate accuracy
                # Ensure the output and labels have compatible dimensions
                #print(f"Output shape before processing: {output.shape}")
                #print(f"Labels shape before processing: {labels.shape}")

                if len(output.shape) > 1 and output.shape[1] > 1:
                    _, predicted = torch.max(output, 1)
                else:
                    predicted = output

                # Convert labels to class indices if they are not already
                if len(labels.shape) > 1 and labels.shape[1] > 1:
                    labels = torch.argmax(labels, dim=1)

                # Print shapes after processing
                #print(f"Output shape after processing: {output.shape}")
                #print(f"Predicted shape: {predicted.shape}")
                #print(f"Labels shape after processing: {labels.shape}")

                # Print values to debug
                #print(f"Output: {output}")
                #print(f"Predicted: {predicted}")
                #print(f"Labels: {labels}")

                correct = (predicted == labels).sum().item()
                accuracy = correct / labels.size(0)
                accuracies.append(accuracy)

                # Print correct predictions and accuracy
                #print(f"Correct predictions: {correct}")
                #print(f"Accuracy: {accuracy}")


                print(
                      f"Epoch [{epoch + 1}/{num_epochs}], "
                      f"Step [{itr + 1}/{len(train_dataset)}], "
                      f"Train Loss: {train_loss:.4f}, "
                      f"Test Loss: {test_loss:.4f}"
                  )

                if not run_recurrent:
                    nump_subs = sub_score.detach().numpy()
                    labels = labels.detach().numpy()
                    print_review(reviews_text[0], nump_subs[0,:,0], nump_subs[0,:,1], labels[0,0], labels[0,1])

                  # saving the model
                torch.save(model, model.name() + ".pth")
      results[hidden_size] = {
        "train_losses": train_losses,
        "test_losses": test_losses,
        "accuracies": accuracies
      }

    return results

In [ ]:
# @title Parameters RNN and GRU
batch_size = 32
output_size = 2
#hidden_size = 64        # to experiment with

run_recurrent = False    # else run Token-wise MLP
use_RNN = False         # otherwise GRU
atten_size = 0          # atten > 0 means using restricted self atten

reload_model = False
num_epochs = 10
learning_rate = 0.001
test_interval = 50

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Loading sataset, use toy = True for obtaining a smaller dataset
train_dataset, test_dataset, num_words, input_size = get_data_set(batch_size)

In [ ]:
# @title Run recurrent and RGU
results = {}
hidden_sizes = [64
                # , 128
                ]
run_model(results, hidden_sizes=hidden_sizes, run_recurrent=run_recurrent, use_RNN=use_RNN, reload_model=reload_model, atten_size=atten_size)
for hidden_size, result in results.items():
    plt.figure()
    plt.plot(result['train_losses'], label='Train Loss')
    plt.plot(result['test_losses'], label='Test Loss')
    plt.plot(result['accuracies'], label='Accuracy')
    plt.title(f'Performance with hidden size of{hidden_size}')
    plt.xlabel('Iterations')
    plt.ylabel('Loss/Accuracy')
    plt.legend()
    plt.show()

# Q2

In [43]:
# @title MLP Module
class ExMLP(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(ExMLP, self).__init__()

        self.ReLU = torch.nn.ReLU()

        # Token-wise MLP network weights
        self.layer1 = MatMul(input_size,hidden_size)
        self.layer2 = MatMul(hidden_size,output_size)
        # self.layer1 = nn.Linear(input_size,hidden_size) # (batch_size,hidden, 2 )
        # self.layer2 = nn.Linear(hidden_size,output_size)


    def name(self):
        return "MLP"

    def forward(self, x):
        '''
        x: (batch_size, sequence, embedded)
        '''

        # Token-wise MLP network implementation
        x = x.clone()
        x = self.layer1(x)
        x = self.ReLU(x)
        x = self.layer2(x)
        x = self.ReLU(x) #(batch_size, seq,2)
        x = torch.mean(x, dim=1)
        x = torch.softmax(x,dim=1)

        return x

    def sub_prediction(self, x):
        """
        x: (batch_size, sequence, embedded)
        return prd, sub_prd
        """
        x = x
        x = self.layer1(x)
        x = self.ReLU(x)
        x = self.layer2(x)
        sub_prd = self.ReLU(x).clone()
        prd = torch.mean(sub_prd, dim=1)
        prd = torch.softmax(prd,dim=1)
        return prd, sub_prd



In [42]:
# @title Run Function for MLP

def run_model_MLP(num_epochs, hidden_sizes,input_size,output_size, reload_model=False, attention=False,lr=1e-3,test_interval=50,print_examples_review=False):
    results = {}
    for hidden_size in hidden_sizes:
        if attention:
            model = ExLRestSelfAtten(input_size, output_size, hidden_size,5)
        else:
            model = ExMLP(input_size, output_size, hidden_size)

        print("Using model: " + model.name())

        if reload_model:
            print("Reloading model")
            model.load_state_dict(torch.load(model.name() + ".pth"))

        # move model to GPU if avilavble
        model.to(DEVICE)

        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.1,step_size_up=5,mode="triangular2")

        train_loss = 1.0
        test_loss = 1.0

        train_losses = []
        test_losses = []
        accuracies = []

      # training steps in which a test step is executed every test_interval

        for epoch in range(num_epochs):
            for itr, (labels, reviews, reviews_text) in enumerate(train_dataset):   # getting training batches
                labels = labels.to(DEVICE)
                reviews = reviews.to(DEVICE)

                output = model(reviews)
                # output = torch.mean(sub_score, 1)
                labels = torch.argmax(labels, dim=1).long()
                # cross-entropy loss
                loss = criterion(output, labels)
                # optimize in training iterations

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                scheduler.step()

                # averaged losses

                train_loss = 0.7 * float(loss.detach()) + 0.3 * train_loss

                if (itr + 1) % test_interval == 0:
                    labels, reviews, reviews_text = next(iter(test_dataset)) # get a test batch:
                    with torch.no_grad():
                        labels = labels.to(DEVICE)
                        reviews = reviews.to(DEVICE)

                        if attention:
                            # MLP + atten
                            output,sub_prd,atten_weights = model.sub_prediction(reviews)
                        else:
                            # MLP
                            output, sub_prd = model.sub_prediction(reviews)

                        # output = torch.mean(sub_score, 1)
                        labels = torch.argmax(labels, dim=1).long()
                        loss = criterion(output, labels)
                        test_loss = 0.7 * float(loss.detach()) + 0.3 * test_loss


                    train_losses.append(train_loss)
                    test_losses.append(test_loss)
                    _, predicted = torch.max(output, 1)


                    correct = (predicted == labels).sum().item()
                    accuracy = correct / labels.size(0)
                    accuracies.append(accuracy)

                    # Print correct predictions and accuracy




                    print(f"""
                    \rHidden size: {hidden_size}
                    \rEpoch [{epoch + 1}/{num_epochs}]
                    \rStep [{itr + 1}/{len(train_dataset)}]
                    \rTrain Loss: {train_loss:.4f}
                    \rTest Loss: {test_loss:.4f}
                    \rCorrect predictions: [{correct}/{labels.size(0)}]
                    \rAccuracy: {accuracy}""")

                    nump_subs = sub_prd.detach()
                    # print(reviews_text[0],nump_subs[0],labels[0])
                    if print_examples_review:
                        print_review(reviews_text[0], nump_subs[0],labels[0])

                    # saving the model
                    torch.save(model, f'{model.name()}_{hidden_size}.pth')
        results[hidden_size] = {
        "train_losses": train_losses,
        "test_losses": test_losses,
        "accuracies": accuracies
    }

    return results

In [ ]:
# @title Parameters MLP
batch_size = 64
input_size = 100
output_size = 2
atten_size = 0          # atten > 0 means using restricted self atten

reload_model = False
num_epochs = 10
learning_rate = 0.001

In [ ]:
# Loading sataset, use toy = True for obtaining a smaller dataset
train_dataset, test_dataset, num_words, input_size = get_data_set(batch_size)

In [ ]:
# @title run
results = {}
hidden_sizes = [128]
results = run_model_MLP(num_epochs, hidden_sizes,input_size,output_size)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=results[32]['test_losses'], name='32 test'))
fig.add_trace(go.Scatter(y=results[32]['train_losses'], name='32 train'))
fig.add_trace(go.Scatter(y=results[64]['test_losses'], name='64 test'))
fig.add_trace(go.Scatter(y=results[64]['train_losses'], name='64 train'))
fig.add_trace(go.Scatter(y=results[128]['test_losses'], name='128 test'))
fig.add_trace(go.Scatter(y=results[128]['train_losses'], name='128 train'))
fig.show()

# Q3:

In [44]:
# @title SelfAttention Module

class ExLRestSelfAtten(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, atten_size):
        super(ExLRestSelfAtten, self).__init__()

        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.atten_size = atten_size
        self.sqrt_hidden_size = np.sqrt(float(hidden_size))
        self.ReLU = torch.nn.ReLU()
        self.softmax = torch.nn.Softmax(dim=-1)
        self.positional_encoding = PositionalEncoding(hidden_size, 2*atten_size+1)

        # Token-wise MLP + Restricted Attention network implementation
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.W_q = nn.Linear(hidden_size, hidden_size, bias=False)
        self.W_k = nn.Linear(hidden_size, hidden_size, bias=False)
        self.W_v = nn.Linear(hidden_size, hidden_size, bias=False)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
            nn.ReLU(),
        )

    def name(self):
        return "MLP_atten"

    def sub_prediction(self, x):
        # x shape: (batch_size, seq_len, input_size)
        x = self.layer1(x)
        x = self.ReLU(x)  # (batch_size, seq_len, hidden_size)

        # Padding and rolling for restricted self-attention
        padded = pad(x, (0, 0, self.atten_size, self.atten_size, 0, 0))
        x_nei = torch.stack([torch.roll(padded, k, dims=1) for k in range(-self.atten_size, self.atten_size+1)], dim=2)
        x_nei = x_nei[:, self.atten_size:-self.atten_size, :, :]
        x_nei = self.positional_encoding(x_nei)

        # Computing q, k, v
        q = self.W_q(x_nei)
        k = self.W_k(x_nei)
        v = self.W_v(x_nei)

        # Attention mechanism
        d = torch.matmul(q, k.transpose(-2, -1)) / self.sqrt_hidden_size
        aw = self.softmax(d)
        x = torch.matmul(aw, v)
        x = torch.mean(x, dim=2)

        # Final MLP layer
        sub_prd = self.mlp(x)
        prd = torch.mean(sub_prd, dim=1)
        prd = torch.softmax(prd, dim=-1)

        return prd, sub_prd, aw

    def forward(self, x):
        return self.sub_prediction(x)[0]

In [36]:
# @title Parameters SelfAtten
batch_size = 64
input_size = 100
output_size = 2
atten_size = 0          # atten > 0 means using restricted self atten

reload_model = False
num_epochs = 5
learning_rate = 0.001

In [37]:
train_dataset, test_dataset, num_words, input_size = get_data_set(batch_size)

In [40]:
# @title run SelfAtten
results = {}
hidden_sizes = [64,32,128]
results = run_model_MLP(num_epochs, hidden_sizes,input_size,output_size,attention=True)

Using model: MLP_atten

Hidden size: 64
Epoch [1/5]
Step [50/469]
Train Loss: 0.6931
Test Loss: 0.7852
Correct predictions: [30/64]
Accuracy: 0.46875

Hidden size: 64
Epoch [1/5]
Step [100/469]
Train Loss: 0.6931
Test Loss: 0.7208
Correct predictions: [31/64]
Accuracy: 0.484375

Hidden size: 64
Epoch [1/5]
Step [150/469]
Train Loss: 0.6931
Test Loss: 0.7014
Correct predictions: [32/64]
Accuracy: 0.5


KeyboardInterrupt: 

In [32]:
fig = go.Figure()
# fig.add_trace(go.Scatter(y=results[128]['test_losses'], name='128 test'))
# fig.add_trace(go.Scatter(y=results[128]['train_losses'], name='128 train'))
# fig.add_trace(go.Scatter(y=results[64]['test_losses'], name='64 test'))
# fig.add_trace(go.Scatter(y=results[64]['train_losses'], name='64 train'))
fig.add_trace(go.Scatter(y=results[32]['test_losses'], name='32 test'))
fig.add_trace(go.Scatter(y=results[32]['train_losses'], name='32 train'))
# fig.add_trace(go.Scatter(y=results[16]['test_losses'], name='16 test'))
# fig.add_trace(go.Scatter(y=results[16]['train_losses'], name='16 train'))

In [72]:
# @title Compare SelfAttention model and Mlp model
atten_m = torch.load("MLP_atten_32.pth")
mlp_m = torch.load("MLP_128.pth")
# train_dataset, test_dataset, num_words, input_size = get_data_set(400)
my_text_dl = create_my_list_dl()
gpt_dl = chatgpt_dl()
compare_model(mlp_m,atten_m,gpt_dl)

In [ ]:
chatgpt_review[27]

'The visual motifs are so frequent, I started seeing them in my breakfast cereal.'